In [49]:
import time
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS
from influxdb_client.client.exceptions import InfluxDBError
from urllib3.exceptions import NewConnectionError
from influxdb_client import Point
from influxdb_client.client.query_api import QueryApi

In [50]:
INFLUX_URL = "http://influxdb2:8086"
INFLUX_TOKEN = "MyInitialAdminToken0=payosno"

client = None
write_api = None

try:
    client = influxdb_client.InfluxDBClient(
        url=INFLUX_URL,
        token=INFLUX_TOKEN,
        org="docs"
    )

    write_api = client.write_api(
        batch=5000,
        write_api=SYNCHRONOUS,
    )
    
except (InfluxDBError, NewConnectionError) as e:
    print("[ERROR] Error al conectar con InfluxDB:")
    print(f"   Detalle: {e}")

# Parte 1: Consultas de filtrado y estructura
## Tarea 1.1 Precio de cierre de Bitcoin

In [51]:
query_api: QueryApi = client.query_api()

query_flux = f"""
from(bucket: "crypto_raw")
 |> range(start: 2020-12-01T00:00:00Z, stop: 2021-01-01T00:00:00Z)
 |> filter(fn: (r) => r["_measurement"] == "daily_quote")
 |> filter(fn: (r) => r["_field"] == "Close")
 |> filter(fn: (r) => r["Symbol"] == "BTC")
"""
results = query_api.query( org="docs",query=query_flux)

for table in results:
    for record in table.records:
        print(f"Tiempo: {record['_time']} | Valor: {record['_value']} | Campo: {record['_field']}")


Tiempo: 2020-12-01 23:59:59+00:00 | Valor: 19633.77044727 | Campo: Close
Tiempo: 2020-12-02 23:59:59+00:00 | Valor: 18801.74359314 | Campo: Close
Tiempo: 2020-12-03 23:59:59+00:00 | Valor: 19205.92540409 | Campo: Close
Tiempo: 2020-12-04 23:59:59+00:00 | Valor: 19446.96642246 | Campo: Close
Tiempo: 2020-12-05 23:59:59+00:00 | Valor: 18698.38527851 | Campo: Close
Tiempo: 2020-12-06 23:59:59+00:00 | Valor: 19154.18059333 | Campo: Close
Tiempo: 2020-12-07 23:59:59+00:00 | Valor: 19343.12879819 | Campo: Close
Tiempo: 2020-12-08 23:59:59+00:00 | Valor: 19191.52946283 | Campo: Close
Tiempo: 2020-12-09 23:59:59+00:00 | Valor: 18320.88478403 | Campo: Close
Tiempo: 2020-12-10 23:59:59+00:00 | Valor: 18553.29972814 | Campo: Close
Tiempo: 2020-12-11 23:59:59+00:00 | Valor: 18263.92925463 | Campo: Close
Tiempo: 2020-12-12 23:59:59+00:00 | Valor: 18051.31998973 | Campo: Close
Tiempo: 2020-12-13 23:59:59+00:00 | Valor: 18806.76647897 | Campo: Close
Tiempo: 2020-12-14 23:59:59+00:00 | Valor: 19144.49

## Tarea 1.2 Volumen total de Etherium

In [52]:
query_api: QueryApi = client.query_api()

query_flux = f"""
from(bucket: "crypto_raw")
  |> range(start: 2021-01-01T00:00:00Z, stop: 2021-07-01T00:00:00Z)
  |> filter(fn: (r) => r._measurement == "daily_quote")
  |> filter(fn: (r) => r.Symbol == "ETH")
  |> filter(fn: (r) => r._field == "Volume")
  |> sum()
"""
results = query_api.query( org="docs",query=query_flux)

for table in results:
    for record in table.records:
        total_volume = record['_value']
        
        formatted_volume = f"{total_volume:,.0f}" 

        print(f"✅ Volumen Total de ETH (Ene-Jun 2021): {formatted_volume}")


✅ Volumen Total de ETH (Ene-Jun 2021): 6,239,381,609,387


 # Parte 2: Agregacion temporal
 ## Tarea 2.1 precio promedio mensual
 

In [ ]:
query_api: QueryApi = client.query_api()

query_flux = f"""
from(bucket: "crypto_raw")
  |> range(start: 0)
  |> filter(fn: (r) => r._measurement == "daily_quote")
  |> filter(fn: (r) => r.Symbol == "BTC")
  |> filter(fn: (r) => r._field == "Close")
  |> aggregateWindow(every: 1mo, fn: mean, createEmpty: false)
"""
results = query_api.query(org="docs", query=query_flux)

for table in results:
    for record in table.records:
        time_str = record['_time'].strftime("%Y-%m")
        avg_price = record['_value']
        formatted_price = f"{avg_price:,.2f}"

        print(f"Mes: {time_str} | Precio Promedio de Cierre: $ {formatted_price}")

Mes: 2013-05 | Precio Promedio de Cierre: $ 139.22
Mes: 2013-06 | Precio Promedio de Cierre: $ 120.29
Mes: 2013-07 | Precio Promedio de Cierre: $ 108.86
Mes: 2013-08 | Precio Promedio de Cierre: $ 90.31
Mes: 2013-09 | Precio Promedio de Cierre: $ 113.04
Mes: 2013-10 | Precio Promedio de Cierre: $ 130.18
Mes: 2013-11 | Precio Promedio de Cierre: $ 156.03
Mes: 2013-12 | Precio Promedio de Cierre: $ 520.51
Mes: 2014-01 | Precio Promedio de Cierre: $ 813.12
Mes: 2014-02 | Precio Promedio de Cierre: $ 841.70
Mes: 2014-03 | Precio Promedio de Cierre: $ 670.84
Mes: 2014-04 | Precio Promedio de Cierre: $ 595.78
Mes: 2014-05 | Precio Promedio de Cierre: $ 461.65
Mes: 2014-06 | Precio Promedio de Cierre: $ 480.99
Mes: 2014-07 | Precio Promedio de Cierre: $ 615.22
Mes: 2014-08 | Precio Promedio de Cierre: $ 619.77
Mes: 2014-09 | Precio Promedio de Cierre: $ 539.69
Mes: 2014-10 | Precio Promedio de Cierre: $ 448.10
Mes: 2014-11 | Precio Promedio de Cierre: $ 365.75
Mes: 2014-12 | Precio Promedio d

## Tarea 2.2 Rango de volatilidad

In [61]:
query = """
data = from(bucket: "crypto_raw")
  |> range(start: 2019-01-01T00:00:00Z, stop: 2020-01-01T00:00:00Z)
  |> filter(fn: (r) => r["_measurement"] == "daily_quote")
  |> filter(fn: (r) => r["Symbol"] == "XLM")
  |> filter(fn: (r) => r["_field"] == "Close")
 
data
  |> aggregateWindow(every: 1w, fn: max, createEmpty: false)
  |> yield(name: "maximo")
 
data
  |> aggregateWindow(every: 1w, fn: min, createEmpty: false)
  |> yield(name: "minimo")
"""
 
results = query_api.query(query)
 
for table in results:
    if len(table.records) > 0:
        tipo_agregacion = table.records[0]["result"]
        print(f"\n--- Resultados de: {tipo_agregacion} ---")
       
        for record in table.records:
            fecha = record.get_time().strftime('%Y-%m-%d')
            valor = record.get_value()
            print(f"Fecha: {fecha} | Valor: {valor:.4f}")


--- Resultados de: minimo ---
Fecha: 2019-01-03 | Valor: 0.1129
Fecha: 2019-01-10 | Valor: 0.1138
Fecha: 2019-01-17 | Valor: 0.1036
Fecha: 2019-01-24 | Valor: 0.1036
Fecha: 2019-01-31 | Valor: 0.0837
Fecha: 2019-02-07 | Valor: 0.0781
Fecha: 2019-02-14 | Valor: 0.0745
Fecha: 2019-02-21 | Valor: 0.0767
Fecha: 2019-02-28 | Valor: 0.0839
Fecha: 2019-03-07 | Valor: 0.0835
Fecha: 2019-03-14 | Valor: 0.0855
Fecha: 2019-03-21 | Valor: 0.1072
Fecha: 2019-03-28 | Valor: 0.1025
Fecha: 2019-04-04 | Valor: 0.1074
Fecha: 2019-04-11 | Valor: 0.1188
Fecha: 2019-04-18 | Valor: 0.1136
Fecha: 2019-04-25 | Valor: 0.1118
Fecha: 2019-05-02 | Valor: 0.0964
Fecha: 2019-05-09 | Valor: 0.0949
Fecha: 2019-05-16 | Valor: 0.0903
Fecha: 2019-05-23 | Valor: 0.1282
Fecha: 2019-05-30 | Valor: 0.1242
Fecha: 2019-06-06 | Valor: 0.1221
Fecha: 2019-06-13 | Valor: 0.1198
Fecha: 2019-06-20 | Valor: 0.1245
Fecha: 2019-06-27 | Valor: 0.1215
Fecha: 2019-07-04 | Valor: 0.1034
Fecha: 2019-07-11 | Valor: 0.0997
Fecha: 2019-07-18

# Parte 3: Manipulacion y Joins
## Tarea 3.1 Cálculo de variacion porcentual diaria

In [70]:
query_api = client.query_api()

query_flux = """
from(bucket: "crypto_raw")
 |> range(start: 2016-01-01T00:00:00Z, stop: 2019-01-01T00:00:00Z)
 |> filter(fn: (r) => r["_measurement"] == "daily_quote")
 |> filter(fn: (r) => r["_field"] == "Close")
 |> filter(fn: (r) => r["Symbol"] == "USDT")
 |> aggregateWindow(every: 1d, fn: last)
 |> difference()
 |> map(fn: (r) => ({ r with porcentaje: r._value * 100.0 }))
"""
results = query_api.query(org="docs", query=query_flux)

for table in results:
    for record in table.records:
        fecha = record["_time"].strftime("%Y-%m-%d")
        print(
            f"Tiempo: {fecha} | Variación porcentual diaria: {record['_value']:.6f}")



Tiempo: 2016-01-03 | Variación porcentual diaria: 0.000000
Tiempo: 2016-01-04 | Variación porcentual diaria: 0.000000
Tiempo: 2016-01-05 | Variación porcentual diaria: 0.000000
Tiempo: 2016-01-06 | Variación porcentual diaria: 0.000000
Tiempo: 2016-01-07 | Variación porcentual diaria: 0.000250
Tiempo: 2016-01-08 | Variación porcentual diaria: -0.000250
Tiempo: 2016-01-09 | Variación porcentual diaria: 0.000000
Tiempo: 2016-01-10 | Variación porcentual diaria: 0.000000
Tiempo: 2016-01-11 | Variación porcentual diaria: 0.000000
Tiempo: 2016-01-12 | Variación porcentual diaria: 0.000000
Tiempo: 2016-01-13 | Variación porcentual diaria: 0.000000
Tiempo: 2016-01-14 | Variación porcentual diaria: 0.000000
Tiempo: 2016-01-15 | Variación porcentual diaria: 0.000460
Tiempo: 2016-01-16 | Variación porcentual diaria: -0.000460
Tiempo: 2016-01-17 | Variación porcentual diaria: 0.000000
Tiempo: 2016-01-18 | Variación porcentual diaria: 0.000390
Tiempo: 2016-01-19 | Variación porcentual diaria: -0.0

## Tarea 3.2 Comparación precios de cierre

In [72]:
query_flux = """
btc_data =
    from(bucket: "crypto_raw")
      |> range(start: 2020-01-01T00:00:00Z, stop: 2021-01-01T00:00:00Z)
      |> filter(fn: (r) => r["_measurement"] == "daily_quote")
      |> filter(fn: (r) => r["_field"] == "Close")
      |> filter(fn: (r) => r["Symbol"] == "BTC")

eth_data =
    from(bucket: "crypto_raw")
      |> range(start: 2020-01-01T00:00:00Z, stop: 2021-01-01T00:00:00Z)
      |> filter(fn: (r) => r["_measurement"] == "daily_quote")
      |> filter(fn: (r) => r["_field"] == "Close")
      |> filter(fn: (r) => r["Symbol"] == "ETH")

join(
    tables: {btc_data: btc_data, eth_data: eth_data},
    on: ["_time"]
)
|> map(fn: (r) => ({
    r with
    ratio: r._value_btc_data / r._value_eth_data
}))
"""

results = query_api.query(org="docs", query=query_flux)

for table in results:
    for record in table.records:
        fecha = record["_time"].strftime("%Y-%m-%d")
        print(f"Fecha: {fecha} | BTC Close: {record['_value_btc_data']:.2f} | ETH Close: {record['_value_eth_data']:.2f} | Ratio BTC/ETH: {record['ratio']:.6f}")


Fecha: 2020-01-01 | BTC Close: 7194.89 | ETH Close: 129.63 | Ratio BTC/ETH: 55.503013
Fecha: 2020-01-02 | BTC Close: 7202.55 | ETH Close: 130.82 | Ratio BTC/ETH: 55.056941
Fecha: 2020-01-03 | BTC Close: 6984.43 | ETH Close: 127.41 | Ratio BTC/ETH: 54.817985
Fecha: 2020-01-04 | BTC Close: 7345.38 | ETH Close: 134.17 | Ratio BTC/ETH: 54.747381
Fecha: 2020-01-05 | BTC Close: 7410.45 | ETH Close: 135.07 | Ratio BTC/ETH: 54.862936
Fecha: 2020-01-06 | BTC Close: 7410.45 | ETH Close: 136.31 | Ratio BTC/ETH: 54.366475
Fecha: 2020-01-07 | BTC Close: 7768.68 | ETH Close: 144.31 | Ratio BTC/ETH: 53.832546
Fecha: 2020-01-08 | BTC Close: 8161.94 | ETH Close: 143.49 | Ratio BTC/ETH: 56.882424
Fecha: 2020-01-09 | BTC Close: 8082.30 | ETH Close: 141.16 | Ratio BTC/ETH: 57.257990
Fecha: 2020-01-10 | BTC Close: 7878.31 | ETH Close: 138.97 | Ratio BTC/ETH: 56.691913
Fecha: 2020-01-11 | BTC Close: 8162.19 | ETH Close: 143.41 | Ratio BTC/ETH: 56.915431
Fecha: 2020-01-12 | BTC Close: 8033.26 | ETH Close: 14